In [25]:
import os

from astroquery.vizier import Vizier
from astroquery.simbad import Simbad

skipDirs = ['.git', 'general', '.vscode']

In [6]:
simbadGaiaIds = {}

for objName in os.listdir(os.getcwd()):
	if os.path.isdir(objName) and objName not in skipDirs:
		simbadResult = Simbad.query_objectids(objName)
		allIds = simbadResult['ID']
		for astroId in allIds:
			if 'Gaia DR3' in astroId:
				simbadGaiaIds[objName] = float(astroId.replace('Gaia DR3 ', ''))
simbadGaiaIds

{'ATO J172.8778+47.9056': 7.88873415117892e+17,
 'CRTS J074502.6+253437': 8.687838929873417e+17,
 'LINEAR_16694484': 1.3849724758727135e+18,
 'LINEAR_44914': 9.342673008441536e+17,
 'TYC4002-2628-1': 1.996670926588668e+18}

In [11]:
# https://stackoverflow.com/questions/38847690/convert-float-to-string-in-positional-format-without-scientific-notation-and-fa#38847691
import decimal

ctx = decimal.Context()
ctx.prec = 40

def float_to_str(f):
    """
    Convert the given float to a string,
    without resorting to scientific notation
    """
    d1 = ctx.create_decimal(repr(f))
    return format(d1, 'f')

{objName: float_to_str(gid) for objName, gid in simbadGaiaIds.items()}

{'ATO J172.8778+47.9056': '788873415117892000',
 'CRTS J074502.6+253437': '868783892987341700',
 'LINEAR_16694484': '1384972475872713500',
 'LINEAR_44914': '934267300844153600',
 'TYC4002-2628-1': '1996670926588668000'}

In [15]:
gaiaSourceInfo = {}

for objName in os.listdir(os.getcwd()):
	if os.path.isdir(objName) and objName not in skipDirs:
		tableListResult = Vizier.query_object(objName, catalog='I/355/gaiadr3') # query Gaia DR3 table
		gaiaSourceInfo[objName] = tableListResult[0]
		print(objName)
		print(tableListResult[0]['Source'])
		print()

ATO J172.8778+47.9056
      Source      
------------------
788870357101176960
788873307730282752
788873415117891968
788873518197107200
788873586903450112
788873651327513728
788874342830828160

CRTS J074502.6+253437
      Source      
------------------
868782101989684352
868782308143103616
868782308143105280
868782312439381248
868783132776676352
868783515029645824
868783579453266048
868783583749761920
868783613813011968
868783618109499776
               ...
868783961706815744
868783991770145536
868783991770371200
868784610245585408
868784614541851008
868784614542237440
868784614542254848
868784644605176064
868784683260503424
868784717621062016
868784816403872128
Length = 38 rows

LINEAR_16694484
       Source      
-------------------
1384971440784499584
1384971479440297472
1384971651238990464
1384971715662586880
1384971715664168576
1384971719958467968
1384971719958640896
1384971754317308928
1384972059259805952
1384972402857181952
1384972402857182080
1384972407153685632
13849724372169

In [16]:
gaiaTable = list(gaiaSourceInfo.values())[0]
gaiaTable.info

<Table length=7>
   name    dtype       unit       format                                                           description                                                           n_bad
--------- ------- ------------- --------- ------------------------------------------------------------------------------------------------------------------------------ -----
  RA_ICRS float64           deg {:15.11f}                                                                                       Right ascension (ICRS) at Ep=2016.0 (ra)     0
  DE_ICRS float64           deg {:15.11f}                                                                                          Declination (ICRS) at Ep=2016.0 (dec)     0
   Source   int64                                                                         Unique source identifier (unique within a particular Data Release) (source_id)     0
e_RA_ICRS float64           mas   {:7.4f}                                                                   

In [31]:
class HRSourceMags:

	def __init__(self, obj_name, gdr3_source_id, g_mag, bp_rp) -> None:
		self.obj_name = obj_name
		self.gdr3_source_id = gdr3_source_id
		self.g_mag = g_mag
		self.bp_rp = bp_rp

	def __str__(self) -> str:
		return f"({self.obj_name}|{str(self.gdr3_source_id)}: G={self.g_mag}, BP-RP={self.bp_rp})"
	
	def __repr__(self) -> str:
		return self.__str__()

objColors: list[HRSourceMags] = []

debugTable: None
for obj, gaiaTable in gaiaSourceInfo.items():
	print(obj, str(simbadGaiaIds[obj]))
	matchedSourceInfo = gaiaTable[gaiaTable['Source'] == int(simbadGaiaIds[obj])]
	try:
		objColors.append(HRSourceMags(obj, float(simbadGaiaIds[obj]), matchedSourceInfo['Gmag'][0], matchedSourceInfo['BP-RP'][0]))
	except Exception as e:
		print('\tERR', e)
		debugTable = gaiaTable
objColors
# debugTable

ATO J172.8778+47.9056 7.88873415117892e+17
CRTS J074502.6+253437 8.687838929873417e+17
LINEAR_16694484 1.3849724758727135e+18
LINEAR_44914 9.342673008441536e+17
TYC4002-2628-1 1.996670926588668e+18
	ERR index 0 is out of bounds for axis 0 with size 0


[(ATO J172.8778+47.9056|7.88873415117892e+17: G=12.934679, BP-RP=1.152715),
 (CRTS J074502.6+253437|8.687838929873417e+17: G=13.584597, BP-RP=1.031323),
 (LINEAR_16694484|1.3849724758727135e+18: G=14.653597, BP-RP=0.650382),
 (LINEAR_44914|9.342673008441536e+17: G=14.774841, BP-RP=1.277842)]